In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import pandas as pd
import numpy as np
from scipy import stats
import glob
import os

# مسیر پوشه شامل فایل‌های Parquet
folder_path = '/content/drive/MyDrive/24may_amin_dataset'  # پوشه شامل فایل‌ها
parquet_files = glob.glob(os.path.join(folder_path, '*.parquet'))

# اصلاح نام ستون‌ها
columns_to_check = ['temperature', 'pressure', 'permeability', 'flow_rate']
z_score_threshold = 3

# ساخت پوشه خروجی برای داده‌های پرت و داده‌های پاک‌شده
outlier_folder = '/content/outliers_files/'
clean_folder = '/content/clean_files/'
os.makedirs(outlier_folder, exist_ok=True)
os.makedirs(clean_folder, exist_ok=True)

# پردازش هر فایل جداگانه
for file_path in parquet_files:
    file_name = os.path.basename(file_path)
    print(f"🔎 پردازش فایل: {file_name}")

    df = pd.read_parquet(file_path)

    # بررسی اگر ستون‌ها موجودند
    if not all(col in df.columns for col in columns_to_check):
        print(f"⚠️ ستون‌های مورد نیاز در {file_name} پیدا نشدند. ادامه به فایل بعدی...")
        continue

    # z-score
    z_scores = np.abs(stats.zscore(df[columns_to_check], nan_policy='omit'))
    outliers_zscore = (z_scores > z_score_threshold)

    # IQR
    outliers_iqr = pd.DataFrame(False, index=df.index, columns=columns_to_check)
    for col in columns_to_check:
        Q1 = df[col].quantile(0.25)
        Q3 = df[col].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        outliers_iqr[col] = (df[col] < lower_bound) | (df[col] > upper_bound)

    # ترکیب نتایج دو روش
    outliers_combined = outliers_zscore | outliers_iqr

    # داده‌های پرت و پاک‌شده
    df_outliers = df[outliers_combined.any(axis=1)].copy()
    df_clean = df[~outliers_combined.any(axis=1)].copy()

    # ذخیره خروجی‌ها با نام اصلی فایل
    outlier_file = os.path.join(outlier_folder, f"outliers_{file_name}")
    clean_file = os.path.join(clean_folder, f"clean_{file_name}")

    df_outliers.to_parquet(outlier_file, index=False)
    df_clean.to_parquet(clean_file, index=False)

    print(f"✅ فایل‌های خروجی ذخیره شدند: {outlier_file}, {clean_file}")

print("🎉 تمام فایل‌ها پردازش شدند.")


🔎 پردازش فایل: Haynesville_50.parquet
✅ فایل‌های خروجی ذخیره شدند: /content/outliers_files/outliers_Haynesville_50.parquet, /content/clean_files/clean_Haynesville_50.parquet
🔎 پردازش فایل: Haynesville_31881.parquet
✅ فایل‌های خروجی ذخیره شدند: /content/outliers_files/outliers_Haynesville_31881.parquet, /content/clean_files/clean_Haynesville_31881.parquet
🔎 پردازش فایل: Haynesville_34068.parquet
✅ فایل‌های خروجی ذخیره شدند: /content/outliers_files/outliers_Haynesville_34068.parquet, /content/clean_files/clean_Haynesville_34068.parquet
🔎 پردازش فایل: Haynesville_81715.parquet
✅ فایل‌های خروجی ذخیره شدند: /content/outliers_files/outliers_Haynesville_81715.parquet, /content/clean_files/clean_Haynesville_81715.parquet
🔎 پردازش فایل: Haynesville_35068.parquet
✅ فایل‌های خروجی ذخیره شدند: /content/outliers_files/outliers_Haynesville_35068.parquet, /content/clean_files/clean_Haynesville_35068.parquet
🔎 پردازش فایل: Haynesville_362.parquet
✅ فایل‌های خروجی ذخیره شدند: /content/outliers_files/ou

In [4]:
import shutil

# فشرده کردن پوشه داده‌های پرت
shutil.make_archive('/content/outliers_files', 'zip', '/content/outliers_files')
# فشرده کردن پوشه داده‌های clean
shutil.make_archive('/content/clean_files', 'zip', '/content/clean_files')


'/content/clean_files.zip'

In [5]:
import shutil

# مسیر پوشه مقصد در گوگل درایو
gdrive_path_outliers = '/content/drive/MyDrive/outliers_files'
gdrive_path_clean = '/content/drive/MyDrive/clean_files'

# کپی کردن پوشه‌ها به درایو
shutil.copytree('/content/outliers_files', gdrive_path_outliers, dirs_exist_ok=True)
shutil.copytree('/content/clean_files', gdrive_path_clean, dirs_exist_ok=True)

print("✅ پوشه‌ها به Google Drive منتقل شدند.")


✅ پوشه‌ها به Google Drive منتقل شدند.
